# Foursquare 

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import sparql_dataframe
import geopandas as gpd
from shapely import wkt
import matplotlib.pyplot as plt

In [2]:
endpoint = "http://localhost:3030/artlas/sparql"

In [3]:
q ="""

PREFIX aat: <http://vocab.getty.edu/aat/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX crmdig: <http://www.ics.forth.gr/isl/CRMdig/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?exhibition ?labelExhibition ?begin ?end ?participants ?labelartist ?addressExhibitionLabel ?Exhibitionwkt ?labeladdressArtist ?addressWKT WHERE {
   
  
  ?exhibition a crm:E7_Activity ;
  	rdfs:label ?labelExhibition ;
  	crm:P2_has_type aat:300054766 ;
   crm:P7_took_place_at ?paris ;
   crm:P4_has_time-span ?timespan ;
   crm:P7_took_place_at ?addressExhibition ;
   crm:P11_had_participant ?participants .
  ?participants rdfs:label ?labelartist .
               
  ?addressExhibition crm:P2_has_type aat:300386983 ;
           rdfs:label ?addressExhibitionLabel ;
           crm:P168_place_is_defined_by ?Exhibitionwkt .
  
  ?timespan crm:P82a_begin_of_the_begin ?begin ;
            crm:P82b_end_of_the_end ?end .
  
   
  ?paris crmdig:L54_is_same-as wd:Q90 .
         
  ?participants crm:P14i_performed ?e13 .
  
  ?e13 crm:P141_assigned ?addressArtist ;
       crm:P140_assigned_attribute_to ?exhibition .
  ?addressArtist rdfs:label ?labeladdressArtist ;
                 crm:P168_place_is_defined_by ?addressWKT .
  
  
}


"""

In [4]:
df = sparql_dataframe.get(endpoint, q)

In [6]:
df['counts'] = df.groupby(['exhibition'])['labelartist'].transform('count')

In [12]:
df["exhibition"].nunique()

39

In [7]:
df.head()

,exhibition,labelExhibition,begin,end,participants,labelartist,addressExhibitionLabel,Exhibitionwkt,labeladdressArtist,addressWKT,counts
0,https://artlas.huma-num.fr/basart/resources/ex...,4me exposition de peinture par M. … (4e impres...,1879-01-01T00:00:00,1879-12-31T23:59:59,https://artlas.huma-num.fr/basart/resources/ag...,Camille Pissarro,avenue de l'Opéra 28,POINT(2.3340511 48.8671836),rue des Trois-Frères 1,POINT(2.3415932 48.8835907),12
1,https://artlas.huma-num.fr/basart/resources/ex...,4me exposition de peinture par M. … (4e impres...,1879-01-01T00:00:00,1879-12-31T23:59:59,https://artlas.huma-num.fr/basart/resources/ag...,Claude Monet,avenue de l'Opéra 28,POINT(2.3340511 48.8671836),rue Vintimille 2,POINT(2.3284023 48.881781),12
2,https://artlas.huma-num.fr/basart/resources/ex...,4me exposition de peinture par M. … (4e impres...,1879-01-01T00:00:00,1879-12-31T23:59:59,https://artlas.huma-num.fr/basart/resources/ag...,Gustave Caillebotte,avenue de l'Opéra 28,POINT(2.3340511 48.8671836),boulevard Haussmann 31,POINT(2.3319986 48.8729841),12
3,https://artlas.huma-num.fr/basart/resources/ex...,4me exposition de peinture par M. … (4e impres...,1879-01-01T00:00:00,1879-12-31T23:59:59,https://artlas.huma-num.fr/basart/resources/ag...,Adolphe-Félix Cals,avenue de l'Opéra 28,POINT(2.3340511 48.8671836),rue Saint-Georges 29,POINT(2.3370123 48.8764072),12
4,https://artlas.huma-num.fr/basart/resources/ex...,4me exposition de peinture par M. … (4e impres...,1879-01-01T00:00:00,1879-12-31T23:59:59,https://artlas.huma-num.fr/basart/resources/ag...,Degas,avenue de l'Opéra 28,POINT(2.3340511 48.8671836),rue Fontaine-Saint-Georges 19 bis,POINT(2.334004 48.8815171),12


In [9]:
df[['ExhibitionLong', 'ExhibitionLat']] = df['Exhibitionwkt'].str.extract(r'POINT\(([^ ]+) ([^ ]+)\)')

In [10]:
df[['addressLong', 'addressLat']] = df['addressWKT'].str.extract(r'POINT\(([^ ]+) ([^ ]+)\)')

In [11]:
df.to_csv('/Users/carboni/Downloads/foursquare.csv', index=False)